In [13]:
# Python program to illustrate the concept 
# of threading 
import threading 
import os 

h = [] 
def task1(x): 
    print(x)
    print("Task 1 assigned to thread: {}".format(threading.current_thread().name)) 
    print("ID of process running task 1: {}".format(os.getpid())) 
    h.append(x)
    
def task2(): 
    print("Task 2 assigned to thread: {}".format(threading.current_thread().name)) 
    print("ID of process running task 2: {}".format(os.getpid())) 
  
if __name__ == "__main__": 
  
    # print ID of current process 
    print("ID of process running main program: {}".format(os.getpid())) 
  
    # print name of main thread 
    print("Main thread name: {}".format(threading.current_thread().name)) 
    l = ['1', '2']
    # creating threads 
    t1 = threading.Thread(target=task1, name='t1', args=(l,) ) 
    t2 = threading.Thread(target=task2, name='t2', )   
  
    # starting threads 
    t1.start() 
    t2.start() 
  
    # wait until all threads finish 
    t1.join() 
    t2.join() 
    print(h)

ID of process running main program: 8107
Main thread name: MainThread
['1', '2']
Task 1 assigned to thread: t1
ID of process running task 1: 8107
Task 2 assigned to thread: t2
ID of process running task 2: 8107
[['1', '2']]


In [1]:
!ls

app.csv			    autoclass_out.tsv	   autoclass.tsv
autoclass-c		    autoclass-report.log   data
autoclass-c-3-3-6.tar.gz.1  autoclass.results-bin  demo_real_scalar.tsv
autoclass.case-data-1	    autoclass.rlog	   multithread.ipynb
autoclass.class-data-1	    autoclass.r-params	   nohup.out
autoclass.db2		    autoclass-run-success  result
autoclass.hd2		    autoclass.search	   resultCal.csv
autoclass.influ-o-data-1    autoclass-search.log   test1.ipynb
autoclass.log		    autoclass.sh	   Untitled.ipynb
autoclass.model		    autoclass.s-params	   wget-log


In [1]:
from pathlib import Path
import sys
import time
import threading

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd
%matplotlib inline
import autoclasswrapper as wrapper
import os 

afterall = {}

def clustering(subfeatures, lock):
    global afterall
    for i in subfeatures:
        runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/" + i
        if not Path(runtimeDir).exists():
            os.mkdir(runtimeDir)
        os.chdir(runtimeDir)
#         if not Path(runtimeDir + "/autoclass-c").exists():
#             !wget https://ti.arc.nasa.gov/m/project/autoclass/autoclass-c-3-3-6.tar.gz
#             !tar zxvf autoclass-c-3-3-6.tar.gz
#             !rm -f autoclass-c-3-3-6.tar.gz
    for i in subfeatures:
        lock.acquire()
        runtimeDir = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/" + i
        os.chdir(runtimeDir)
        temp = ":" + runtimeDir + "/autoclass-c"
        os.environ['PATH'] += temp

        # Create object to prepare dataset.
        clust = wrapper.Input()

        # Load datasets from tsv files.
        pathData = "/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/" + i + ".tsv"
        clust.add_input_data(pathData, "real location")

        # Prepare input data:
        # - create a final dataframe
        # - merge datasets if multiple inputs
        clust.prepare_input_data()

        # Create files needed by AutoClass.
        clust.create_db2_file()
        clust.create_hd2_file()
        clust.create_model_file()
        clust.create_sparams_file(max_duration = 0)
        clust.create_rparams_file()
        
        lock.release()
#         !rm -f autoclass-run-* *.results-bin


#         # Search autoclass in path.
#         wrapper.search_autoclass_in_path()

#         # Create object to run AutoClass.
#         run = wrapper.Run()

#         # Prepare run script.
#         run.create_run_file()

#         # Run AutoClass.
#         run.run()
        
#         timer = 0
#         step = 2
#         while not Path(runtimeDir + "/autoclass-run-success").exists():
#             timer += step
#             sys.stdout.write("\r")
#             sys.stdout.write(f"Time: {timer} sec.")
#             sys.stdout.flush()
#             time.sleep(step)
#             if Path("autoclass-run-failure").exists():
#                 print("error")
#                 break
#         if Path(runtimeDir + "/autoclass-run-success").exists():
#             results = wrapper.Output()
#             results.extract_results()
#             results.aggregate_input_data()
#             results.write_cdt()
#             results.write_cdt(with_proba=True)
#             results.write_class_stats()
#             results.write_dendrogram()
#             result = pd.read_csv(runtimeDir + 'autoclass_out.tsv', encoding="utf-8", header=0, sep='\t', index_col=0)
#             label =  pd.read_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/ProtocolName.tsv', encoding='utf-8', header=0, sep='\t', index_col=0)

#             label['cluster'] = result['main-class']
#             label.to_csv('app.csv', index = True, header=True)


#             listH = []
#             listApp = []
#             listCluster = []
#             def fun(x):
#                 listCluster.append(x['cluster'].values[0])
#                 listH.append(x['ProtocolName'].value_counts().max()/x.shape[0])
#                 listApp.append(x['ProtocolName'].value_counts().idxmax())


#             label.groupby('cluster').apply(fun)

#             H = sum(listH)/len(listH)
#             print(listH)
#             print(listApp)
#             print(listCluster)
#             print(H)
            
#             dict = {'Cluster': listCluster, 'App': listApp, 'H': listH}

#             saveResult = pd.DataFrame(dict)

#             saveResult.to_csv(runtimeDir + 'resultCal.csv', index = True, header=True)

#             afterall.update({i:H})
#         lock.release()
    #    lock.acquire()
    #    H.append(i)
    #    haha += 1
    #    lock.release()

if __name__ == "__main__":
    
    features = ['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets',
       'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min',
       'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min',
       'Bwd_Packet_Length_Mean', 'Bwd_Packet_Length_Std', 'Flow_Bytes_s',
       'Flow_Packets_s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max',
       'Flow_IAT_Min', 'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std',
       'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean',
       'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags',
       'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Packets_s', 'Bwd_Packets_s',
       'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count',
       'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count',
       'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Flag_Count', 'Down_Up_Ratio',
       'Average_Packet_Size', 'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size',
       'Fwd_Header_Length_1', 'Fwd_Avg_Bytes_Bulk', 'Fwd_Avg_Packets_Bulk',
       'Fwd_Avg_Bulk_Rate', 'Bwd_Avg_Bytes_Bulk', 'Bwd_Avg_Packets_Bulk',
       'Bwd_Avg_Bulk_Rate', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes',
       'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean',
       'Idle_Std', 'Idle_Max', 'Idle_Min']
    datacore = []
    chunkSize = len(features) / 7
    datacore.append(features[0: int(chunkSize)])
    datacore.append(features[int(chunkSize) : int(2*chunkSize)])
    datacore.append(features[int(2*chunkSize): int(3*chunkSize)])
    datacore.append(features[int(3*chunkSize): int(4*chunkSize)])
    datacore.append(features[int(4*chunkSize): int(5*chunkSize)])
    datacore.append(features[int(5*chunkSize): int(6*chunkSize)])
    datacore.append(features[int(6*chunkSize): len(features)])
    
    listThread = []
    
    lock = threading.Lock()
    # creating threads
    for data in datacore:
        i = 1
        listThread.append(threading.Thread(target=clustering, name='t' + str(i), args=(data,lock,)))
        i = i + 1
                          
    for i in listThread:
        i.start()
    for i in listThread:
        i.join()
    
# round1 = pd.DataFrame(afterall)
# round1.to_csv('/home/chuongdv/Documents/SourceCode/MachineLearning/resl.csv', index = True, header=True)

2020-04-26 20:11:49 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Flow_Duration.tsv' as 'real location' with error 0.01
2020-04-26 20:11:49 INFO     Detected encoding: ascii
2020-04-26 20:11:49 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:49 DEBUG    Checking column names
2020-04-26 20:11:49 DEBUG    Index name 'index'
2020-04-26 20:11:49 DEBUG    Column name 'Flow_Duration'
2020-04-26 20:11:49 INFO     Checking data format
2020-04-26 20:11:49 INFO     Column 'Flow_Duration'
2020-04-26 20:11:49 INFO     count    4.916000e+03
2020-04-26 20:11:49 INFO     mean     2.700050e+07
2020-04-26 20:11:49 INFO     std      4.043878e+07
2020-04-26 20:11:49 INFO     min      1.000000e+00
2020-04-26 20:11:49 INFO     50%      1.005040e+06
2020-04-26 20:11:49 INFO     max      1.200000e+08
2020-04-26 20:11:49 INFO     ---
2020-04-26 20:11:49 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:49 INFO     Searching for m

2020-04-26 20:11:49 INFO     min          0.000000
2020-04-26 20:11:49 INFO     50%        223.000000
2020-04-26 20:11:49 INFO     max      24616.000000
2020-04-26 20:11:49 INFO     ---
2020-04-26 20:11:49 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:49 INFO     Searching for missing values
2020-04-26 20:11:49 INFO     No missing values found
2020-04-26 20:11:49 INFO     Writing autoclass.db2 file
2020-04-26 20:11:49 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:49 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:49 INFO     Writing .hd2 file
2020-04-26 20:11:49 INFO     Writing .model file
2020-04-26 20:11:49 INFO     Writing .s-params file
2020-04-26 20:11:49 INFO     Writing .r-params file
2020-04-26 20:11:49 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Fwd_Packet_Length_Min.tsv' as 'real location' with error 0.01
2020-04-26 20:11:49 INFO     Detected encoding

2020-04-26 20:11:50 INFO     Detected encoding: ascii
2020-04-26 20:11:50 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:50 DEBUG    Checking column names
2020-04-26 20:11:50 DEBUG    Index name 'index'
2020-04-26 20:11:50 DEBUG    Column name 'Fwd_Avg_Bulk_Rate'
2020-04-26 20:11:50 INFO     Checking data format
2020-04-26 20:11:50 INFO     Column 'Fwd_Avg_Bulk_Rate'
2020-04-26 20:11:50 INFO     count    4916.0
2020-04-26 20:11:50 INFO     mean        0.0
2020-04-26 20:11:50 INFO     std         0.0
2020-04-26 20:11:50 INFO     min         0.0
2020-04-26 20:11:50 INFO     50%         0.0
2020-04-26 20:11:50 INFO     max         0.0
2020-04-26 20:11:50 INFO     ---
2020-04-26 20:11:50 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:50 INFO     Searching for missing values
2020-04-26 20:11:50 INFO     No missing values found
2020-04-26 20:11:50 INFO     Writing autoclass.db2 file
2020-04-26 20:11:50 INFO     If any, missing values will be encoded as '?'
20

2020-04-26 20:11:50 INFO     Writing autoclass.db2 file
2020-04-26 20:11:50 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:50 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:50 INFO     Writing .hd2 file
2020-04-26 20:11:50 INFO     Writing .model file
2020-04-26 20:11:50 INFO     Writing .s-params file
2020-04-26 20:11:50 INFO     Writing .r-params file
2020-04-26 20:11:50 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Subflow_Bwd_Packets.tsv' as 'real location' with error 0.01
2020-04-26 20:11:50 INFO     Detected encoding: ascii
2020-04-26 20:11:50 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:50 DEBUG    Checking column names
2020-04-26 20:11:50 DEBUG    Index name 'index'
2020-04-26 20:11:50 DEBUG    Column name 'Subflow_Bwd_Packets'
2020-04-26 20:11:50 INFO     Checking data format
2020-04-26 20:11:50 INFO     Column 'Subflow_Bwd_Packets'
2020-04-26 20:11:50 INFO     count  

2020-04-26 20:11:50 DEBUG    Column name 'ACK_Flag_Count'
2020-04-26 20:11:50 INFO     Checking data format
2020-04-26 20:11:50 INFO     Column 'ACK_Flag_Count'
2020-04-26 20:11:50 INFO     count    4916.000000
2020-04-26 20:11:50 INFO     mean        0.489626
2020-04-26 20:11:50 INFO     std         0.499943
2020-04-26 20:11:50 INFO     min         0.000000
2020-04-26 20:11:50 INFO     50%         0.000000
2020-04-26 20:11:50 INFO     max         1.000000
2020-04-26 20:11:50 INFO     ---
2020-04-26 20:11:50 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:50 INFO     Searching for missing values
2020-04-26 20:11:50 INFO     No missing values found
2020-04-26 20:11:50 INFO     Writing autoclass.db2 file
2020-04-26 20:11:50 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:50 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:50 INFO     Writing .hd2 file
2020-04-26 20:11:50 INFO     Writing .model file
2020-04-26 20:11:50 IN

...

2020-04-26 20:11:52 DEBUG    Checking column names
2020-04-26 20:11:52 DEBUG    Index name 'index'
2020-04-26 20:11:52 DEBUG    Column name 'Bwd_IAT_Min'
2020-04-26 20:11:52 INFO     Checking data format
2020-04-26 20:11:52 INFO     Column 'Bwd_IAT_Min'
2020-04-26 20:11:52 INFO     count    4.916000e+03
2020-04-26 20:11:52 INFO     mean     7.836932e+05
2020-04-26 20:11:52 INFO     std      6.233174e+06
2020-04-26 20:11:52 INFO     min      0.000000e+00
2020-04-26 20:11:52 INFO     50%      0.000000e+00
2020-04-26 20:11:52 INFO     max      1.103628e+08
2020-04-26 20:11:52 INFO     ---
2020-04-26 20:11:52 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:52 INFO     Searching for missing values
2020-04-26 20:11:52 INFO     No missing values found
2020-04-26 20:11:52 INFO     Writing autoclass.db2 file
2020-04-26 20:11:52 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:52 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:52

2020-04-26 20:11:52 INFO     Writing .hd2 file
2020-04-26 20:11:52 INFO     Writing .model file
2020-04-26 20:11:52 INFO     Writing .s-params file
2020-04-26 20:11:52 INFO     Writing .r-params file
2020-04-26 20:11:52 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Bwd_Header_Length.tsv' as 'real location' with error 0.01
2020-04-26 20:11:52 INFO     Detected encoding: ascii
2020-04-26 20:11:52 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:52 DEBUG    Checking column names
2020-04-26 20:11:52 DEBUG    Index name 'index'
2020-04-26 20:11:52 DEBUG    Column name 'Bwd_Header_Length'
2020-04-26 20:11:52 INFO     Checking data format
2020-04-26 20:11:52 INFO     Column 'Bwd_Header_Length'
2020-04-26 20:11:52 INFO     count    4.916000e+03
2020-04-26 20:11:52 INFO     mean     4.407626e+03
2020-04-26 20:11:52 INFO     std      6.647737e+04
2020-04-26 20:11:52 INFO     min      0.000000e+00
2020-04-26 20:11:52 INFO     50%      1.5

2020-04-26 20:11:52 INFO     std       326.443389
2020-04-26 20:11:52 INFO     min         0.000000
2020-04-26 20:11:52 INFO     50%        77.923077
2020-04-26 20:11:52 INFO     max      3615.797282
2020-04-26 20:11:52 INFO     ---
2020-04-26 20:11:52 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:52 INFO     Searching for missing values
2020-04-26 20:11:52 INFO     No missing values found
2020-04-26 20:11:52 INFO     Writing autoclass.db2 file
2020-04-26 20:11:52 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:52 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:52 INFO     Writing .hd2 file
2020-04-26 20:11:52 INFO     Writing .model file
2020-04-26 20:11:52 INFO     Writing .s-params file
2020-04-26 20:11:52 INFO     Writing .r-params file
2020-04-26 20:11:52 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Packet_Length_Std.tsv' as 'real location' with error 0.01
202

2020-04-26 20:11:53 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/act_data_pkt_fwd.tsv' as 'real location' with error 0.01
2020-04-26 20:11:53 INFO     Detected encoding: ascii
2020-04-26 20:11:53 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:53 DEBUG    Checking column names
2020-04-26 20:11:53 DEBUG    Index name 'index'
2020-04-26 20:11:53 DEBUG    Column name 'act_data_pkt_fwd'
2020-04-26 20:11:53 INFO     Checking data format
2020-04-26 20:11:53 INFO     Column 'act_data_pkt_fwd'
2020-04-26 20:11:53 INFO     count     4916.000000
2020-04-26 20:11:53 INFO     mean        99.035598
2020-04-26 20:11:53 INFO     std        792.090847
2020-04-26 20:11:53 INFO     min          0.000000
2020-04-26 20:11:53 INFO     50%          2.000000
2020-04-26 20:11:53 INFO     max      22009.000000
2020-04-26 20:11:53 INFO     ---
2020-04-26 20:11:53 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:53 INFO     Search

2020-04-26 20:11:53 INFO     Final dataframe has 4916 lines and 2 columns
2020-04-26 20:11:53 INFO     Searching for missing values
2020-04-26 20:11:53 INFO     No missing values found
2020-04-26 20:11:53 INFO     Writing autoclass.db2 file
2020-04-26 20:11:53 INFO     If any, missing values will be encoded as '?'
2020-04-26 20:11:53 DEBUG    Writing autoclass.tsv file [for later use]
2020-04-26 20:11:53 INFO     Writing .hd2 file
2020-04-26 20:11:53 INFO     Writing .model file
2020-04-26 20:11:53 INFO     Writing .s-params file
2020-04-26 20:11:53 INFO     Writing .r-params file
2020-04-26 20:11:53 INFO     Reading data file '/home/chuongdv/Documents/SourceCode/MachineLearning/exp/data/features/Idle_Mean.tsv' as 'real location' with error 0.01
2020-04-26 20:11:53 INFO     Detected encoding: ascii
2020-04-26 20:11:53 INFO     Found 4916 rows and 2 columns
2020-04-26 20:11:53 DEBUG    Checking column names
2020-04-26 20:11:53 DEBUG    Index name 'index'
2020-04-26 20:11:53 DEBUG    Col

In [87]:
print(len(H))
print(len(M))
print(len(k))
if Path("/home/chuongdv/Documents/SourceCode/MachineLearning/exp/haha").exists():
    print("hihi")
print(len(hic))

77
7
0
2


In [54]:
features = ['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets',
       'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min',
       'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min',
       'Bwd_Packet_Length_Mean', 'Bwd_Packet_Length_Std', 'Flow_Bytes_s',
       'Flow_Packets_s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max',
       'Flow_IAT_Min', 'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std',
       'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean',
       'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags',
       'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Packets_s', 'Bwd_Packets_s',
       'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count',
       'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count',
       'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Flag_Count', 'Down_Up_Ratio',
       'Average_Packet_Size', 'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size',
       'Fwd_Header_Length_1', 'Fwd_Avg_Bytes_Bulk', 'Fwd_Avg_Packets_Bulk',
       'Fwd_Avg_Bulk_Rate', 'Bwd_Avg_Bytes_Bulk', 'Bwd_Avg_Packets_Bulk',
       'Bwd_Avg_Bulk_Rate', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes',
       'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean',
       'Idle_Std', 'Idle_Max', 'Idle_Min']
datacore = []
chunkSize = len(features) / 7
datacore.append(features[0: int(chunkSize)])
datacore.append(features[int(chunkSize) : int(2*chunkSize)])
datacore.append(features[int(2*chunkSize): int(3*chunkSize)])
datacore.append(features[int(3*chunkSize): int(4*chunkSize)])
datacore.append(features[int(4*chunkSize): int(5*chunkSize)])
datacore.append(features[int(5*chunkSize): int(6*chunkSize)])
datacore.append(features[int(6*chunkSize): len(features)])

In [55]:
for i in datacore:
    print(i)

['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min']
['Bwd_Packet_Length_Mean', 'Bwd_Packet_Length_Std', 'Flow_Bytes_s', 'Flow_Packets_s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max', 'Flow_IAT_Min', 'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std']
['Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean', 'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags', 'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags']
['Fwd_Header_Length', 'Bwd_Header_Length', 'Fwd_Packets_s', 'Bwd_Packets_s', 'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean', 'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count', 'SYN_Flag_Count']
['RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count', 'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Flag_Count', 'Down_Up_Rati

app.csv			    autoclass_out.tsv	   autoclass.tsv
autoclass-c		    autoclass-report.log   data
autoclass-c-3-3-6.tar.gz.1  autoclass.results-bin  demo_real_scalar.tsv
autoclass.case-data-1	    autoclass.rlog	   multithread.ipynb
autoclass.class-data-1	    autoclass.r-params	   nohup.out
autoclass.db2		    autoclass-run-success  result
autoclass.hd2		    autoclass.search	   resultCal.csv
autoclass.influ-o-data-1    autoclass-search.log   test1.ipynb
autoclass.log		    autoclass.sh	   Untitled.ipynb
autoclass.model		    autoclass.s-params	   wget-log


In [2]:
!ls

autoclass.db2  autoclass.model	   autoclass.s-params
autoclass.hd2  autoclass.r-params  autoclass.tsv


In [3]:
!ls ..


ACK_Flag_Count		Down_Up_Ratio		Fwd_URG_Flags
act_data_pkt_fwd	ECE_Flag_Count		Idle_Max
Active_Max		FIN_Flag_Count		Idle_Mean
Active_Mean		Flow_Bytes_s		Idle_Min
Active_Min		Flow_Duration		Idle_Std
Active_Std		Flow_IAT_Max		Init_Win_bytes_backward
Average_Packet_Size	Flow_IAT_Mean		Init_Win_bytes_forward
Avg_Bwd_Segment_Size	Flow_IAT_Min		Max_Packet_Length
Avg_Fwd_Segment_Size	Flow_IAT_Std		Min_Packet_Length
Bwd_Avg_Bulk_Rate	Flow_Packets_s		min_seg_size_forward
Bwd_Avg_Bytes_Bulk	Fwd_Avg_Bulk_Rate	Packet_Length_Mean
Bwd_Avg_Packets_Bulk	Fwd_Avg_Bytes_Bulk	Packet_Length_Std
Bwd_Header_Length	Fwd_Avg_Packets_Bulk	Packet_Length_Variance
Bwd_IAT_Max		Fwd_Header_Length	PSH_Flag_Count
Bwd_IAT_Mean		Fwd_Header_Length_1	RST_Flag_Count
Bwd_IAT_Min		Fwd_IAT_Max		Subflow_Bwd_Bytes
Bwd_IAT_Std		Fwd_IAT_Mean		Subflow_Bwd_Packets
Bwd_IAT_Total		Fwd_IAT_Min		Subflow_Fwd_Bytes
Bwd_Packet_Length_Max	Fwd_IAT_Std		Subflow_Fwd_Packets
Bwd_Packet_Length_Mean	Fwd_IAT_Total		SYN_Flag_Count
Bwd_Packet_Length_

In [4]:
!pwd

/home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/Idle_Min


In [14]:
!touch /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 

In [16]:
!echo "xref_class_report_att_list = 0, 1\nreport_mode = \"data\"\ncomment_data_headers_p = true" >> /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params 

In [20]:
!cp /home/chuongdv/Documents/SourceCode/MachineLearning/exp/result/autoclass.r-params .

In [18]:
    features = ['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets',
       'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets',
       'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min',
       'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std',
       'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min',
       'Bwd_Packet_Length_Mean', 'Bwd_Packet_Length_Std', 'Flow_Bytes_s',
       'Flow_Packets_s', 'Flow_IAT_Mean', 'Flow_IAT_Std', 'Flow_IAT_Max',
       'Flow_IAT_Min', 'Fwd_IAT_Total', 'Fwd_IAT_Mean', 'Fwd_IAT_Std',
       'Fwd_IAT_Max', 'Fwd_IAT_Min', 'Bwd_IAT_Total', 'Bwd_IAT_Mean',
       'Bwd_IAT_Std', 'Bwd_IAT_Max', 'Bwd_IAT_Min', 'Fwd_PSH_Flags',
       'Bwd_PSH_Flags', 'Fwd_URG_Flags', 'Bwd_URG_Flags', 'Fwd_Header_Length',
       'Bwd_Header_Length', 'Fwd_Packets_s', 'Bwd_Packets_s',
       'Min_Packet_Length', 'Max_Packet_Length', 'Packet_Length_Mean',
       'Packet_Length_Std', 'Packet_Length_Variance', 'FIN_Flag_Count',
       'SYN_Flag_Count', 'RST_Flag_Count', 'PSH_Flag_Count', 'ACK_Flag_Count',
       'URG_Flag_Count', 'CWE_Flag_Count', 'ECE_Flag_Count', 'Down_Up_Ratio',
       'Average_Packet_Size', 'Avg_Fwd_Segment_Size', 'Avg_Bwd_Segment_Size',
       'Fwd_Header_Length_1', 'Fwd_Avg_Bytes_Bulk', 'Fwd_Avg_Packets_Bulk',
       'Fwd_Avg_Bulk_Rate', 'Bwd_Avg_Bytes_Bulk', 'Bwd_Avg_Packets_Bulk',
       'Bwd_Avg_Bulk_Rate', 'Subflow_Fwd_Packets', 'Subflow_Fwd_Bytes',
       'Subflow_Bwd_Packets', 'Subflow_Bwd_Bytes', 'Init_Win_bytes_forward',
       'Init_Win_bytes_backward', 'act_data_pkt_fwd', 'min_seg_size_forward',
       'Active_Mean', 'Active_Std', 'Active_Max', 'Active_Min', 'Idle_Mean',
       'Idle_Std', 'Idle_Max', 'Idle_Min']
    datacore = []
    chunkSize = len(features) / 7
    datacore.append(features[0: int(chunkSize)])
    datacore.append(features[int(chunkSize) : int(2*chunkSize)])
    datacore.append(features[int(2*chunkSize): int(3*chunkSize)])
    datacore.append(features[int(3*chunkSize): int(4*chunkSize)])
    datacore.append(features[int(4*chunkSize): int(5*chunkSize)])
    datacore.append(features[int(5*chunkSize): int(6*chunkSize)])
    datacore.append(features[int(6*chunkSize): len(features)])

In [19]:
print(datacore[0])

['Flow_Duration', 'Total_Fwd_Packets', 'Total_Backward_Packets', 'Total_Length_of_Fwd_Packets', 'Total_Length_of_Bwd_Packets', 'Fwd_Packet_Length_Max', 'Fwd_Packet_Length_Min', 'Fwd_Packet_Length_Mean', 'Fwd_Packet_Length_Std', 'Bwd_Packet_Length_Max', 'Bwd_Packet_Length_Min']
